In [1]:
import pandas as pd 

In [5]:
df = pd.read_csv('../../AssistanceLLM/AssistanceLLM/ft_data_new/baseline.csv')

In [6]:
prompts = df.prompt.values

In [12]:
from chronological import main

In [19]:
MODEL = 'babbage:ft-david-krueger-research-group-2022-09-13-12-07-43'

async def request_batch_completions(prompts, model_name, max_requests=5, batch_size=20):
    """
    Request a batch of completions from fine tuned GPT3 model.
    """
    
    if len(prompts) > 20:
        raise ValueError('The number of prompts to process in a batch must not exceed 20.')

    response = None
    r = 1
    while response is None:
        if r == max_requests:
            print('Maximum number of requests reached.')
            # break
        try:
            response = openai.Completion.create(
                model=model_name,
                prompt=batch_prompts,
                temperature=0.0,
                max_tokens=64,
                top_p=0,
                frequency_penalty=0,
                presence_penalty=0,
                stop=[" ###"]
            )
            completions = [x['text'].strip() for x in response['choices']]

    
        except RateLimitError:
            r += 1
            print('Error while loading model')
            continue

    return completions

async def logic():
    batch_size = 3
    #for i in tqdm(range(0, len(prompts), batch_size)):
    #    batch_prompts = prompts[i:i + batch_size]
    completion = await request_batch_completions(batch_prompts, MODEL)
    print('Completion Response: {0}'.format(completion))

In [22]:
# you can name this function anything you want, the name "logic" is arbitrary
async def logic():
    # you call the Chronology functions, awaiting the ones that are marked await
    prompt = read_prompt('example_prompt')
    completion = await cleaned_completion(prompt, max_tokens=100, engine="davinci", temperature=0.5, top_p=1, frequency_penalty=0.2, stop=["\n\n"])

    print('Completion Response: {0}'.format(completion))
    
    # you can also run whatever you want in this function
    for i in range(4):
        print("hello")


# invoke the Chronology main fn to run the async logic
main(logic)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [1]:
import json

In [4]:
def js_r(filename: str):
    with open(filename) as f_in:
        return json.load(f_in)


def get_qa_data(paragraph) -> list:
    out = []
    for q_data in paragraph['qas']:
        if not q_data['is_impossible']:
            q = q_data['question']
            a = [q_data['answers'][i]['text'] for i in range(len(q_data['answers']))]
            if len(a) > 1:
                a = list(set(a))
                # if len(a) > 1:
                #     print(a)
            a = '; '.join(a)  # possible answers are separated with ";"
            out.append((q, a))
            # out.append(make_qa_prompt(q, a))
    return out


def get_flat_data(json_data) -> list:
    out = []
    for topical_data in json_data['data']:
        for paragraph_with_qs in topical_data['paragraphs']:
            out.append([paragraph_with_qs['context']] + get_qa_data(paragraph_with_qs))
    return out


In [7]:
data = js_r('dev-v2.0.json')

In [15]:
data_train = js_r('train-v2.0.json')

In [13]:
dev_topics = []
for topical_data in data['data']:
    dev_topics.append(topical_data['title'])

In [16]:
train_topics = []
for topical_data in data_train['data']:
    train_topics.append(topical_data['title'])

In [18]:
set(train_topics) & set(dev_topics)

set()

In [19]:
train_topics[:10]

['Beyoncé',
 'Frédéric_Chopin',
 'Sino-Tibetan_relations_during_the_Ming_dynasty',
 'IPod',
 'The_Legend_of_Zelda:_Twilight_Princess',
 'Spectre_(2015_film)',
 '2008_Sichuan_earthquake',
 'New_York_City',
 'To_Kill_a_Mockingbird',
 'Solar_energy']

In [20]:
dev_topics

['Normans',
 'Computational_complexity_theory',
 'Southern_California',
 'Sky_(United_Kingdom)',
 'Victoria_(Australia)',
 'Huguenot',
 'Steam_engine',
 'Oxygen',
 '1973_oil_crisis',
 'European_Union_law',
 'Amazon_rainforest',
 'Ctenophora',
 'Fresno,_California',
 'Packet_switching',
 'Black_Death',
 'Geology',
 'Pharmacy',
 'Civil_disobedience',
 'Construction',
 'Private_school',
 'Harvard_University',
 'Jacksonville,_Florida',
 'Economic_inequality',
 'University_of_Chicago',
 'Yuan_dynasty',
 'Immune_system',
 'Intergovernmental_Panel_on_Climate_Change',
 'Prime_number',
 'Rhine',
 'Scottish_Parliament',
 'Islamism',
 'Imperialism',
 'Warsaw',
 'French_and_Indian_War',
 'Force']